# Deploy pre-trained SpaCy model using S2i

In this notebook, we load a SpaCy based NLP model from S3 storage in-memory and draw predictions from it.

We will then use [s2i](https://github.com/openshift/source-to-image) to operationalize this model and deploy it on OpenShift. This demo is based on Michael Clifford's ODSC demo [notebook](https://github.com/MichaelClifford/example-model-s2i-notebook/blob/master/model.ipynb) and is a specific use-case where we do not wish to train the model in notebook, rather load it in-memory and deploy it using s2i.

## requirements

The first convention to follow is declaring your model's requirements as a list of lists in a variable called requirements. The s2i builder will use these to generate a `requirements.txt` file, which it will install while building an image. This step is optional, but it is necessary if your model will depend on any libraries.

In [1]:
requirements = [["spacy", "3.0.6"], ["boto3", "1.9.205"]]

## load model from s3

The s2i build process will execute every cell in the notebook in order.

In [2]:
import spacy
import os
import boto3
import json
# from dotenv import load_dotenv, find_dotenv #while running locally

s2i will not have access to local directories, hence we will load the serialized SpaCy model from an S3 bucket.

For the deployed application to be able to access these environment variables, we need to add them to the Build as Environment Variables. To add secure credentials to the build, you can first define secrets containing the key and value and import them into the build.

In [17]:
# load_dotenv(find_dotenv()) # use to locate credentials while running notebook locally

In [4]:
my_bucket = "DH-SECURE-OCHATTER"
conn = boto3.client(service_name='s3',
        aws_access_key_id= os.environ.get("s3-access-key"),
        aws_secret_access_key= os.environ.get("s3-secret-key"),
        endpoint_url='https://s3.upshift.redhat.com/')

To load a SpaCy model, essentially the `nlp` object, we need to translate the contents into a byte like object by serializing it and then de-serialize it to load it to use it in our workflow.

The two main objects that are needed to effectively deserialize a SpaCy model are the model `config` and `nlp` object.
The [SpaCy documentation](https://spacy.io/usage/saving-loading) lists the functions that can be used to serialize and desialize the objects for eg:
```python
config = nlp.config
bytes_data = nlp.to_bytes()
```

We have already saved the `config` and `bytes_data` object onto S3 and in this notebook, we will load those from S3, re-create an `nlp` object and use to draw predictions from.

In [5]:
bytes_data = conn.get_object(Bucket=my_bucket, Key = 'model_deploy/model_in_bytes')

In [6]:
bytes_data

{'ResponseMetadata': {'RequestId': 'tx00000000000001a8c2612-0060c3951d-60765d10-default',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-length': '15831120',
   'accept-ranges': 'bytes',
   'last-modified': 'Wed, 09 Jun 2021 19:22:31 GMT',
   'x-rgw-object-type': 'Normal',
   'etag': '"1a191bd2ae70da9ba51204b2b31d7649-2"',
   'x-amz-request-id': 'tx00000000000001a8c2612-0060c3951d-60765d10-default',
   'content-type': 'binary/octet-stream',
   'date': 'Fri, 11 Jun 2021 16:53:49 GMT',
   'connection': 'Keep-Alive',
   'strict-transport-security': 'max-age=31536000; includeSubDomains; preload'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2021, 6, 9, 19, 22, 31, tzinfo=tzutc()),
 'ContentLength': 15831120,
 'ETag': '"1a191bd2ae70da9ba51204b2b31d7649-2"',
 'ContentType': 'binary/octet-stream',
 'Metadata': {},
 'Body': <botocore.response.StreamingBody at 0x7f16ee85b550>}

In [7]:
config = conn.get_object(Bucket=my_bucket, Key = 'model_deploy/config.json')

In [8]:
config

{'ResponseMetadata': {'RequestId': 'tx00000000000000f0a1c31-0060c3951d-652e5ed3-default',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-length': '5017',
   'accept-ranges': 'bytes',
   'last-modified': 'Wed, 09 Jun 2021 19:22:31 GMT',
   'x-rgw-object-type': 'Normal',
   'etag': '"fecc39263771ebd53d9bd453ff3da01a"',
   'x-amz-request-id': 'tx00000000000000f0a1c31-0060c3951d-652e5ed3-default',
   'content-type': 'binary/octet-stream',
   'date': 'Fri, 11 Jun 2021 16:53:49 GMT',
   'connection': 'Keep-Alive',
   'strict-transport-security': 'max-age=31536000; includeSubDomains; preload'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2021, 6, 9, 19, 22, 31, tzinfo=tzutc()),
 'ContentLength': 5017,
 'ETag': '"fecc39263771ebd53d9bd453ff3da01a"',
 'ContentType': 'binary/octet-stream',
 'Metadata': {},
 'Body': <botocore.response.StreamingBody at 0x7f16ee85bf98>}

In [9]:
data = bytes_data['Body'].read()

In [10]:
type(data)

bytes

In [11]:
config_data = config['Body'].read()
json_content = json.loads(config_data)

In [12]:
type(json_content)

dict

Now we will re-create the nlp object from the `config` and `bytes_data` files.

In [13]:
lang_cls = spacy.util.get_lang_class(json_content["nlp"]["lang"])
nlp = lang_cls.from_config(json_content)
nlp.from_bytes(data)

Example prediction from the `nlp` object.

In [14]:
doc = nlp("IBM is an organization")
[(ent.text, ent.label_) for ent in doc.ents]

[('IBM', 'ORG')]

## Predictor and Validator

Given the trained model, you simply need to provide two functions:

`predictor`, which will make a single prediction from a single sample, this can be customized as per the output you need to return from the model,
and  
`validator`, which will return True if a single sample is of the correct type.

In [15]:
def predictor(feed_text):
    
    doc = nlp(feed_text)
    return [(ent.text, ent.label_) for ent in doc.ents]

`validator` is optional, but it will make your model service easier to use. If you don't provide one, your model service will accept any input, which will likely lead to confusing error messages (i.e., crashes somewhere in the predictor) if your model service is called with bogus input.

In [16]:
def validator(x):
    
    return type(x) == str